# Prohledávání hyperparametrů pro model TinyViT nad datasetem CIFAR10 v původní i augmentované podobě

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem CIFAR10 pro model TinyViT. Hyperparametry jsou hledány pro původní i augmentovaný dataset pro normální trénink i destilaci.

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou z variant.

## Import knihoven a základní nastavení

In [ ]:
from transformers import Trainer, AutoModelForImageClassification
from torch.utils.data import ConcatDataset
import optuna
import torch
import math
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [ ]:
base.reset_seed()

Ověření dostupnosti GPU.

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení datasetu a aplikace základních a augmentačních transformací.

In [5]:
DATASET = "cifar10"

In [ ]:
transform = base.base_transforms()

test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)

In [7]:
augment_transform = base.aug_transforms()
train_aug = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

Provedení filtrace augmentovaného datasetu dle popsaného mechanismu.

In [8]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku jsou dvě epochy, maximální sedm epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [ ]:
num_epochs = 7
batch_size = 128

In [ ]:
data_length = len(train)
min_r = math.ceil(data_length/batch_size)*2
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad původním datasetem
Definice hledaných hyperparametrů a jejich rozmezí.

In [12]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [ ]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [15]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/_hp-search", logging_dir=f"~/logs/{DATASET}/_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice získání studentského modelu.

In [ ]:
def get_model():
    return AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=10, ignore_mismatched_sizes=True)

Konfigurace trenéra pro jednotlivé tréninky. 

In [17]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_model()
)
  

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [18]:
best_base = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base",
    n_trials=150
)

[I 2025-03-29 16:18:44,733] A new study created in memory with name: Base


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.442700,0.264957,0.911700,0.919823,0.911798,0.911894
2,0.138400,0.190786,0.937400,0.939950,0.937880,0.937399
3,0.074700,0.146533,0.955900,0.956957,0.955881,0.956239
4,0.042500,0.150831,0.958400,0.958616,0.958541,0.958336


[I 2025-03-29 16:29:31,752] Trial 0 pruned. 


...

[I 2025-03-30 10:53:28,623] Trial 148 finished with value: 0.9741444675782933 and parameters: {'learning_rate': 9.806964711146234e-05, 'weight_decay': 0.005, 'warmup_steps': 31}. Best is trial 67 with value: 0.9746290627725797.


Trial 149 with params: {'learning_rate': 0.00015367446969151156, 'weight_decay': 0.008, 'warmup_steps': 24}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.440300,0.202930,0.933300,0.938917,0.933727,0.933899
2,0.097600,0.138907,0.952700,0.954045,0.953010,0.952865
3,0.046900,0.120470,0.964000,0.964423,0.964160,0.964183
4,0.022700,0.143753,0.962200,0.962246,0.962467,0.962233


[I 2025-03-30 10:59:03,079] Trial 149 pruned. 


In [19]:
print(best_base)

BestRun(run_id='67', objective=0.9746290627725797, hyperparameters={'learning_rate': 9.777098843358782e-05, 'weight_decay': 0.007, 'warmup_steps': 24}, run_summary=None)


In [20]:
base.reset_seed()

## Prohledávání s destilací nad původním datasetem
Konfigurace jednotlivých tréninků.

In [21]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/-KD_hp-search", logging_dir=f"~/logs/{DATASET}/-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [22]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [23]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky.

In [24]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_model()
)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [25]:
best_distill = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-30 10:59:03,800] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.355700,0.212412,0.930600,0.935032,0.930799,0.931365
2,0.175400,0.188130,0.948100,0.949182,0.948338,0.948282


[I 2025-03-30 11:01:53,132] Trial 0 pruned. 


...

[I 2025-03-31 03:38:44,507] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 7.376314367140723e-05, 'weight_decay': 0.01, 'warmup_steps': 12, 'lambda_param': 0.7000000000000001, 'temperature': 4.5}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.377000,0.187315,0.946900,0.949938,0.947197,0.947298
2,0.150600,0.162884,0.960900,0.961098,0.961091,0.960959
3,0.128600,0.151853,0.968100,0.968263,0.968244,0.968219
4,0.118700,0.150022,0.968300,0.968796,0.968439,0.968466
5,0.113900,0.144931,0.971100,0.971403,0.971213,0.971239
6,0.111600,0.143044,0.970900,0.971185,0.971047,0.971051
7,0.110100,0.142287,0.971500,0.971642,0.971659,0.971619


[I 2025-03-31 03:48:35,832] Trial 149 finished with value: 0.9716190451722241 and parameters: {'learning_rate': 7.376314367140723e-05, 'weight_decay': 0.01, 'warmup_steps': 12, 'lambda_param': 0.7000000000000001, 'temperature': 4.5}. Best is trial 32 with value: 0.9764120594095738.


In [26]:
print(best_distill)

BestRun(run_id='32', objective=0.9764120594095738, hyperparameters={'learning_rate': 0.00013553561983282748, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 1.0, 'temperature': 3.0}, run_summary=None)


In [27]:
base.reset_seed()

Přepočet kroků s ohledem na změnu velikosti datasetu.

In [ ]:
data_length = len(train_combo)
min_r = math.ceil(data_length/batch_size)*2
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [28]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/-aug_hp-search", logging_dir=f"~/logs/{DATASET}/-aug_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [29]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [30]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [31]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_model()
)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [32]:
best_base_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-head",
    n_trials=150
)

[I 2025-03-31 03:48:36,446] A new study created in memory with name: Base-head


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.388300,0.221293,0.926000,0.929451,0.926255,0.926541
2,0.131200,0.166291,0.947400,0.947967,0.947885,0.947319


[I 2025-03-31 03:53:01,180] Trial 0 pruned. 


...

[I 2025-04-01 01:11:37,935] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0001397082047711854, 'weight_decay': 0.009000000000000001, 'warmup_steps': 18}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.371000,0.153391,0.948200,0.949074,0.948247,0.948450
2,0.093500,0.132794,0.956600,0.956868,0.956851,0.956540


[I 2025-04-01 01:16:02,133] Trial 149 pruned. 


In [33]:
print(best_base_aug)

BestRun(run_id='71', objective=0.9750019868944989, hyperparameters={'learning_rate': 0.00013900432881088528, 'weight_decay': 0.007, 'warmup_steps': 13}, run_summary=None)


In [34]:
base.reset_seed()

## Prohledávání s destilací nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/-aug-KD_hp-search", logging_dir=f"~/logs/{DATASET}/-aug-KD_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [38]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_model()
)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [39]:
best_distill_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-01 01:16:02,812] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.330400,0.220720,0.930000,0.932340,0.930202,0.929863
2,0.193100,0.202633,0.938600,0.940075,0.938903,0.938784


[I 2025-04-01 01:20:25,975] Trial 0 pruned. 


...

[I 2025-04-01 21:56:59,597] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00011872523215251572, 'weight_decay': 0.009000000000000001, 'warmup_steps': 24, 'lambda_param': 0.8, 'temperature': 3.0}


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([10, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.334600,0.179448,0.951100,0.952146,0.951246,0.951254
2,0.171200,0.163259,0.961100,0.961534,0.961335,0.961152
3,0.148200,0.154251,0.968400,0.968782,0.968551,0.968601
4,0.137900,0.147217,0.971000,0.971261,0.971172,0.971161
5,0.132700,0.144941,0.971100,0.971304,0.971274,0.971274


[I 2025-04-01 22:07:59,936] Trial 149 pruned. 


In [40]:
print(best_distill_aug)

BestRun(run_id='14', objective=0.9760268868449631, hyperparameters={'learning_rate': 0.00014946504427538972, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.9, 'temperature': 3.0}, run_summary=None)


In [41]:
print("Best random init training score: ", best_base)
print("Best random init distilation trianing score: ", best_distill)
print("Best pretrained (head only) training score: ", best_base_aug)
print("Best pretrained distilation (head only) training score: ",best_distill_aug)

Best random init training score:  BestRun(run_id='67', objective=0.9746290627725797, hyperparameters={'learning_rate': 9.777098843358782e-05, 'weight_decay': 0.007, 'warmup_steps': 24}, run_summary=None)
Best random init distilation trianing score:  BestRun(run_id='32', objective=0.9764120594095738, hyperparameters={'learning_rate': 0.00013553561983282748, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 1.0, 'temperature': 3.0}, run_summary=None)
Best pretrained (head only) training score:  BestRun(run_id='71', objective=0.9750019868944989, hyperparameters={'learning_rate': 0.00013900432881088528, 'weight_decay': 0.007, 'warmup_steps': 13}, run_summary=None)
Best pretrained distilation (head only) training score:  BestRun(run_id='14', objective=0.9760268868449631, hyperparameters={'learning_rate': 0.00014946504427538972, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.9, 'temperature': 3.0}, run_summary=None)
